In [1]:
import pandas as pd
import numpy as np

In [2]:
import unicodedata

In [3]:
# Función para eliminar acentos
def eliminar_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto)
        texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    return texto

In [4]:
def comparar_listas(lista1, lista2):
    """
    Compara dos listas y devuelve los elementos comunes y los elementos únicos en cada lista.

    Args:
    lista1 (list): La primera lista a comparar.
    lista2 (list): La segunda lista a comparar.

    Returns:
    tuple: Una tupla que contiene tres listas:
        - Elementos comunes en ambas listas.
        - Elementos únicos en la primera lista.
        - Elementos únicos en la segunda lista.
    """
    # Convertir las listas a conjuntos para facilitar la comparación
    set1 = set(lista1)
    set2 = set(lista2)

    # Encontrar elementos comunes
    comunes = set1.intersection(set2)

    # Encontrar elementos únicos en cada lista
    unicos_lista1 = set1.difference(set2)
    unicos_lista2 = set2.difference(set1)

    # Convertir los resultados a listas
    comunes = list(comunes)
    unicos_lista1 = list(unicos_lista1)
    unicos_lista2 = list(unicos_lista2)

    return comunes, unicos_lista1, unicos_lista2

In [5]:
df_flats = pd.read_excel('./data/sample-flats-madrid-synthetic-coords.xlsx', engine = 'openpyxl')

In [6]:
df_gpd = pd.read_excel('./data/frongdf.xlsx', engine = 'openpyxl')

#### Crear diccionario de barrios

In [7]:
df_flats['LOCATIONNAME'] = df_flats['LOCATIONNAME'].str.lower().apply(eliminar_acentos)

In [8]:
df_gpd['NOMBRE'] = df_gpd['NOMBRE'].str.lower().apply(eliminar_acentos) 

In [9]:
flats_lst = df_flats.LOCATIONNAME.unique().tolist()
gpd_lst = df_gpd.NOMBRE.unique().tolist()

In [10]:
comun, unico_flats, unico_gpd = comparar_listas(flats_lst, gpd_lst)

In [11]:
# Encontrar la longitud máxima de las listas
max_length = max(len(unico_flats), len(unico_gpd), len(comun))

# Rellenar las listas más cortas con NaN
unico_flats.extend([np.nan] * (max_length - len(unico_flats)))
unico_gpd.extend([np.nan] * (max_length - len(unico_gpd)))
comun.extend([np.nan] * (max_length - len(comun)))

# Crear un diccionario a partir de las listas
data = {
    'comun': comun,
    'unico_flats': unico_flats,
    'unico_gpd': unico_gpd
}

# Crear el DataFrame
df_cols = pd.DataFrame(data)

#### Traducir columna de flats al glosario de barrios.

In [12]:
df_diccionario = pd.read_excel('./data/dicc_gdp_flat.xlsx', engine = 'openpyxl')

In [13]:
df_diccionario.head(1)

,gpd,flats
0,abrantes,abrantes


In [14]:
# Cambio todos los barrios que aparecen en flats con los datos de gdp, para ello creo el diccionario a partir del df_diccionario
# asegurar que la columna que serán las claves no tiene nulos.
df_filtered = df_diccionario.dropna(subset=['flats'])
dict_flats2gpd = df_filtered.set_index('flats')['gpd'].to_dict()

In [15]:
df_flats['LOCATIONNAME'] = df_flats['LOCATIONNAME'].map(dict_flats2gpd).fillna(df_flats['LOCATIONNAME'])

#### Ahora empezamos a realizar agrupaciones

In [16]:
def get_median_df_byLOCATIONNAME(feature, df):
    median_feature = df.groupby('LOCATIONNAME')[feature].median()
    df_median_feature = median_feature.reset_index()
    df_median_feature.columns = ['LOCATIONNAME', f"{feature}"]
    return df_median_feature

In [17]:
df_gpd.columns

Index(['NOMBRE', 'BARRIO_MAY', 'BARRIO_MT'], dtype='object')

In [18]:
get_median_df_byLOCATIONNAME('PRICE', df_flats)

,LOCATIONNAME,PRICE
0,abrantes,141000.0
1,acacias,333000.0
2,adelfas,370000.0
3,aeropuerto,120000.0
4,aguilas,153000.0
...,...,...
131,ventas,176000.0
132,vinateros,174500.0
133,virgen del cortijo - manoteras,330500.0
134,vista alegre,154000.0


In [ ]:
median_bathnumber = df_flats.groupby('LOCATIONNAME')['BATHNUMBER'].median()

# Convertimos el resultado en un DataFrame
df_median_bathnumber = median_bathnumber.reset_index()

# Renombramos las columnas para mayor claridad
df_median_bathnumber.columns = ['LOCATIONNAME', 'MEDIAN_BATHNUMBER']

In [ ]:
df_flats.columns